In [1]:
import argparse
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from dataclasses import dataclass
from joblib import Parallel, delayed
from tqdm import tqdm

In [2]:
%load_ext autoreload
%autoreload 2

from model import GModel
from myutils import roc_auc, translate_result, filter_target
from load_data import load_data
from optimizer import Optimizer
from sampler import NewSampler

In [3]:
@dataclass
class Args:
    device: str = "cuda" if torch.cuda.is_available() else "cpu"
    data: str = "ctrp"
    n_jobs: int = 6 # ← ここをコア数に応じて調整
    lr: float = 5e-4
    epochs: int = 1000

In [4]:
args = Args()
target_option = "drug"  # "cell" か "drug"

In [5]:
# データ読み込み
res, drug_feature, exprs, mut, cna, null_mask = load_data(args)

response matrix (res) shape: (470, 807)
exprs shape: (1089, 19851)
mut shape: (1089, 1667)
cna shape: (1090, 23316)
807
exprs shape: (807, 19851)
mut shape: (807, 1667)
cna shape: (807, 23316)
drug_feature shape: (470, 920)
response matrix (res) shape: (807, 470)
null_mask shape: (807, 470)


In [6]:
def run_single_model(exprs, cna, mut, drug_feature, res_mat, null_mask, target_dim, target_index, args, seed):
    sampler = NewSampler(res_mat, null_mask, target_dim, target_index)
    val_labels = sampler.test_data[sampler.test_mask]

    model = GModel(
        adj_mat=sampler.train_data.float(),
        gene=exprs,
        cna=cna,
        mutation=mut,
        sigma=2,
        k=11,
        iterates=3,
        feature_drug=drug_feature,
        n_hid1=192,
        n_hid2=36,
        alpha=5.74,
        device=args.device,
    )
    opt = Optimizer(
        model=model,
        train_data=sampler.train_data,
        test_data=sampler.test_data,
        test_mask=sampler.test_mask,
        train_mask=sampler.train_mask,
        evaluate_fun=roc_auc,
        lr=args.lr,
        epochs=args.epochs,
        device=args.device,
    ).to(args.device)
    _, true_data, predict_data = opt()
    return true_data, predict_data

In [7]:
# ターゲット次元と統計
target_dim = 0 if target_option == "cell" else 1
samples = res.shape[target_dim]
cell_sum = np.sum(res.values, axis=1)
drug_sum = np.sum(res.values, axis=0)

In [8]:
# 並列実行用ラッパー
def process_target(seed, target_index):
    try:
        return run_single_model(
            exprs=exprs,
            cna=cna,
            mut=mut,
            drug_feature=drug_feature,
            res_mat=res.values,
            null_mask=null_mask,
            target_dim=target_dim,
            target_index=target_index,
            args=args,
            seed=seed,
        )
    except Exception as e:
        print(f"❌ Failed at target {target_index}: {e}")
        return None

In [9]:
# 結果格納用
true_data_s = pd.DataFrame()
predict_data_s = pd.DataFrame()
skipped_targets = []
passed_targets = []

# スキップチェック
for target_index in range(samples):
    label_vec = res.iloc[target_index] if target_dim == 0 else res.iloc[:, target_index]
    passed, reason, pos, neg, total = filter_target(label_vec)

    if passed:
        passed_targets.append(target_index)
    else:
        skipped_targets.append((target_index, reason, pos, neg, total))

# スキップ情報表示
print(f"\n🚫 Skipped Targets: {len(skipped_targets)}")
for idx, reason, pos, neg, total in skipped_targets:
    print(f"Target {idx}: skipped because {reason} (total={total}, pos={pos}, neg={neg})")

# 並列実行（max_njobs を args.n_jobs に変更可能）
results = Parallel(n_jobs=args.n_jobs)(
    delayed(process_target)(seed, target_index)
    for seed, target_index in enumerate(tqdm(passed_targets, desc=f"MOFGCN ({args.data} - {target_option})"))
)

# 結果の統合（None を除外）
for r in results:
    if r is not None:
        true_data, pred_data = r
        true_data_s = pd.concat([true_data_s, translate_result(true_data)], ignore_index=True)
        predict_data_s = pd.concat([predict_data_s, translate_result(pred_data)], ignore_index=True)


🚫 Skipped Targets: 36
Target 2: skipped because few_total_samples (total=4, pos=2, neg=2)
Target 21: skipped because few_total_samples (total=3, pos=1, neg=2)
Target 40: skipped because few_total_samples (total=5, pos=3, neg=2)
Target 41: skipped because few_total_samples (total=8, pos=5, neg=3)
Target 45: skipped because few_total_samples (total=6, pos=1, neg=5)
Target 48: skipped because few_total_samples (total=9, pos=4, neg=5)
Target 52: skipped because few_total_samples (total=1, pos=1, neg=0)
Target 54: skipped because few_total_samples (total=2, pos=1, neg=1)
Target 56: skipped because few_total_samples (total=9, pos=4, neg=5)
Target 58: skipped because few_total_samples (total=9, pos=4, neg=5)
Target 60: skipped because few_total_samples (total=2, pos=1, neg=1)
Target 63: skipped because few_total_samples (total=3, pos=1, neg=2)
Target 68: skipped because few_total_samples (total=3, pos=1, neg=2)
Target 83: skipped because few_total_samples (total=2, pos=1, neg=1)
Target 84: s

MOFGCN (ctrp - drug):   1%|▏         | 6/434 [00:00<00:08, 49.93it/s]/users/4/inoue019/DRBenchmark/MOFGCN/model.py:47: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:203.)
  gene = torch.from_numpy(gene).to(device)
/users/4/inoue019/DRBenchmark/MOFGCN/model.py:47: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internall

epoch:   0 loss:0.757741 auc:0.5375
epoch:  20 loss:0.422271 auc:0.9500
epoch:  40 loss:0.377267 auc:0.9625
epoch:  60 loss:0.357410 auc:0.9500
epoch:  80 loss:0.337258 auc:0.9375
epoch: 100 loss:0.313532 auc:0.9375
epoch: 120 loss:0.283645 auc:0.9375
epoch: 140 loss:0.244958 auc:0.9375
epoch: 160 loss:0.216882 auc:0.9375
epoch: 180 loss:0.198974 auc:0.9375
Fit finished.
epoch:   0 loss:0.798014 auc:0.5000
epoch:  20 loss:0.441246 auc:0.5204
epoch:  40 loss:0.379826 auc:0.5204
epoch:  60 loss:0.362243 auc:0.5357
epoch:  80 loss:0.347705 auc:0.5459
epoch: 100 loss:0.327051 auc:0.5459
epoch: 120 loss:0.304272 auc:0.5510
epoch: 140 loss:0.276990 auc:0.5969
epoch: 160 loss:0.243280 auc:0.5714
epoch: 180 loss:0.216516 auc:0.5765
epoch: 200 loss:0.199469 auc:0.5612
epoch: 220 loss:0.187908 auc:0.5612
epoch: 240 loss:0.179105 auc:0.5306
Fit finished.
epoch:   0 loss:0.773756 auc:0.4701
epoch:  20 loss:0.470179 auc:0.8496
epoch:  40 loss:0.384805 auc:0.8700
epoch:  60 loss:0.367078 auc:0.8802


epoch: 140 loss:0.266050 epoch:   0 loss:0.768452 auc:0.4681
epoch:  20 loss:0.403405 auc:0.8570
epoch:  40 loss:0.376355 auc:0.8786
epoch:  60 loss:0.358608 auc:0.8750
epoch:  80 loss:0.341248 auc:0.8688
epoch: 100 loss:0.317393 auc:0.8431
epoch: 120 loss:0.288822 auc:0.8251
epoch: 140 loss:0.252713 auc:0.8302
epoch: 160 loss:0.221772 auc:0.8513
epoch: 180 loss:0.203051 auc:0.8709
epoch: 200 loss:0.188677 auc:0.8729
Fit finished.
epoch:   0 loss:0.813357 auc:0.4279
epoch:  20 loss:0.466004 auc:0.8758
epoch:  40 loss:0.384726 auc:0.9069
epoch:  60 loss:0.365521 auc:0.9091
epoch:  80 loss:0.352748 auc:0.9180
epoch: 100 loss:0.335282 auc:0.9069
epoch: 120 loss:0.314325 auc:0.8714
epoch: 140 loss:0.289809 auc:0.8182
epoch: 160 loss:0.255789 auc:0.8160
epoch: 180 loss:0.223725 auc:0.8271
Fit finished.
epoch:   0 loss:0.756911 auc:0.6575
epoch:  20 loss:0.397238 auc:0.9382
epoch:  40 loss:0.373354 auc:0.9410
epoch:  60 loss:0.355804 auc:0.9403
epoch:  80 loss:0.337466 auc:0.9403
epoch: 100 

epoch: 400 epoch:   0 loss:0.718217 auc:0.4226
epoch:  20 loss:0.407712 auc:0.9107
epoch:  40 loss:0.375095 auc:0.9152
epoch:  60 loss:0.359603 auc:0.9182
epoch:  80 loss:0.342078 auc:0.9256
epoch: 100 loss:0.315146 auc:0.9286
epoch: 120 loss:0.283720 auc:0.9182
epoch: 140 loss:0.244912 auc:0.8899
epoch: 160 loss:0.215096 auc:0.8973
epoch: 180 loss:0.196958 auc:0.8943
Fit finished.
epoch:   0 loss:0.814188 auc:0.5200
epoch:  20 loss:0.462728 auc:0.9467
epoch:  40 loss:0.382432 auc:0.9400
epoch:  60 loss:0.366079 auc:0.9467
epoch:  80 loss:0.355212 auc:0.9467
epoch: 100 loss:0.339795 auc:0.9200
epoch: 120 loss:0.317262 auc:0.9067
epoch: 140 loss:0.289564 auc:0.9267
epoch: 160 loss:0.253899 auc:0.9200
epoch: 180 loss:0.225043 auc:0.9267
Fit finished.
epoch:   0 loss:0.769072 auc:0.3485
epoch:  20 loss:0.449894 auc:0.9324
epoch:  40 loss:0.380423 auc:0.9310
epoch:  60 loss:0.365621 auc:0.9372
epoch:  80 loss:0.352072 auc:0.9420
epoch: 100 loss:0.332366 auc:0.9409
epoch: 120 loss:0.310099 

...

MOFGCN (ctrp - drug):  95%|█████████▌| 414/434 [02:54<00:07,  2.52it/s]

loss:0.846221 auc:0.5770
epoch:  20 loss:0.508268 auc:0.8114
epoch:  40 loss:0.388629 auc:0.8214
epoch:  60 loss:0.369472 auc:0.8399
epoch:  80 loss:0.356060 auc:0.8525
epoch: 100 loss:0.341744 auc:0.8595
epoch: 120 loss:0.322329 auc:0.8536
epoch: 140 loss:0.302250 auc:0.8364
epoch: 160 loss:0.276077 auc:0.8141
epoch: 180 loss:0.243713 auc:0.7806
Fit finished.
epoch:   0 loss:0.858111 auc:0.5168
epoch:  20 loss:0.430189 auc:0.6852
epoch:  40 loss:0.377205 auc:0.6674
epoch:  60 loss:0.361313 auc:0.6714
epoch:  80 loss:0.349290 auc:0.6751
epoch: 100 loss:0.333487 auc:0.6753
epoch: 120 loss:0.311071 auc:0.6704
epoch: 140 loss:0.282804 auc:0.6547
epoch: 160 loss:0.248496 auc:0.6352
epoch: 180 loss:0.219174 auc:0.6744
Fit finished.
epoch:   0 loss:0.784705 auc:0.3835
epoch:  20 loss:0.458006 auc:0.9649
epoch:  40 loss:0.380138 auc:0.9649
epoch:  60 loss:0.364490 auc:0.9654
epoch:  80 loss:0.350652 auc:0.9677
epoch: 100 loss:0.328691 auc:0.9666
epoch: 120 loss:0.300420 auc:0.9638
epoch: 140 

MOFGCN (ctrp - drug):  97%|█████████▋| 420/434 [02:56<00:05,  2.59it/s]

loss:0.811057 auc:0.4375
epoch:  20 loss:0.481498 auc:0.6042
epoch:  40 loss:0.381588 auc:0.6042
epoch:  60 loss:0.364926 auc:0.6042
epoch:  80 loss:0.352271 auc:0.6042
epoch: 100 loss:0.335183 auc:0.6458
epoch: 120 loss:0.312531 auc:0.6458
epoch: 140 loss:0.289000 auc:0.6042
epoch: 160 loss:0.259628 auc:0.5625
epoch: 180 loss:0.229083 auc:0.5000
Fit finished.
epoch:   0 loss:0.809204 auc:0.5553
epoch:  20 loss:0.441672 auc:0.9449
epoch:  40 loss:0.380709 auc:0.9539
epoch:  60 loss:0.363359 auc:0.9658
epoch:  80 loss:0.348567 auc:0.9718
epoch: 100 loss:0.328558 auc:0.9718
epoch: 120 loss:0.308504 auc:0.9720
epoch: 140 loss:0.281214 auc:0.9550
epoch: 160 loss:0.246110 auc:0.8971
epoch: 180 loss:0.217782 auc:0.8948
Fit finished.
epoch:   0 loss:0.819860 auc:0.3900
epoch:  20 loss:0.413141 auc:0.8475
epoch:  40 loss:0.373859 auc:0.8638
epoch:  60 loss:0.355261 auc:0.8745
epoch:  80 loss:0.335757 auc:0.8815
epoch: 100 loss:0.309122 auc:0.8764
epoch: 120 loss:0.271219 auc:0.8588
epoch: 140 

MOFGCN (ctrp - drug): 100%|██████████| 434/434 [03:02<00:00,  2.38it/s]


In [10]:
# Save
true_path = f"mofgcn_true_{args.data}_{target_option}.csv"
pred_path = f"mofgcn_pred_{args.data}_{target_option}.csv"
true_data_s.to_csv(true_path, index=False)
predict_data_s.to_csv(pred_path, index=False)

print(f"\n✅ Done. Results saved to:\n  - {true_path}\n  - {pred_path}")


✅ Done. Results saved to:
  - mofgcn_true_ctrp_drug.csv
  - mofgcn_pred_ctrp_drug.csv
